## PLANTILLA

In [46]:
# **************************************** Variables Fuente ***********************************************************
nombre_archivo = 'INFORME_ENDEUDAMIENTO_SISTEMA_TRIM_II_2024_def.xlsx'  # Nombre del archivo, ej:'Archivo Original.xlsx'
periodo = 'Q20242' # Periodo del archivo, ej:M202102

path = 'D:/Fuentes/Planos/Cifin-Empresas/' # Carpeta raiz de la fuente
nombre_diccionario = 'dict_meta_riesgos-creditos_cifin-empresas_cifin_v6.xlsx' # Nombre del diccionario de datos

file_path = path + 'Originales/' + nombre_archivo
file_path_dict = 'D:/Fuentes/Planos/Diccionarios/' + nombre_diccionario

rename_columns = { 'clasificacion_capas_(junio_2023)':'clasificacion_del_cliente'
                 }

###### read_file

In [47]:
import pandas as pd
import numpy as np
import sys
import json
import os
import warnings
from calendar import monthrange
from datetime import datetime
pd.set_option('max_columns', None)
pd.set_option('max_rows',50)
warnings.filterwarnings("ignore", category=DeprecationWarning)

#Función carga archivo en df con columnas en minuscula
def read_file(file_path:str) -> pd.DataFrame:
    if os.path.splitext(file_path)[1] == '.xlsx':
        df = pd.read_excel(file_path,
            skiprows = None,
            #decimal = ',',
            dtype = 'str'
            )
    else:
        df = pd.read_csv(file_path,
            sep = ';',
            decimal = ',',
            encoding = 'utf-8',
#            encoding = 'ISO-8859-1',
            low_memory = False,
#            quoting = 3,
            error_bad_lines = False,
            dtype = 'str'
            )
    list_columns = df.columns.to_list()
    list_new_columns = []
    for item in list_columns:
        item = str(item)
        item = item.replace('-', '')
        item = item.replace('%', '')
        item = item.replace(':', '')
        item = item.replace('/', '')
        item = item.replace('.', '')
        item = item.replace(',', '')
        item = item.replace('\n', ' ')
        item = item.replace('\t', ' ')
        item = item.replace('  ', ' ')
        item = item.strip()
        item = item.replace(' ', '_')
        item = item.replace('__', '_')
        item = item.lower()
        item = item.replace('á', 'a')
        item = item.replace('é', 'e')
        item = item.replace('í', 'i')
        item = item.replace('ó', 'o')
        item = item.replace('ú', 'u')
        list_new_columns.append(item)
    df.columns = list_new_columns
    df = df.rename(columns = rename_columns)
    #df = df.drop(['de_occidente_TOTAL', 'de_occidente_com_me', 'de_occidente_cant_obl_com', 'de_occidente_cant_obl_total', 'de_occidente_cant_obl_leas', 'de_occidente_com_ml','Unnamed 211'], axis=1)
    return df

###### generar_load

In [48]:
def generar_load(file_path:str):   
    df = pd.read_csv(file_path, 
                  sep = '|', 
                  decimal = ',',
                  encoding = 'utf-8',
                  low_memory = False,
                  quoting = 3,
                  error_bad_lines = False,
                ) 
    entity = 'BOCC'
    filename = os.path.basename(file_path)
    split = filename.split('_')
    domain = split[0]
    subdomain = split[1]
    source = split[2]
    periodo = split[3].split('.')[0]
    len_period = len(periodo)
    first_letter = periodo[0:1]
    extencion = split[3].split('.')[1]
    bucket_name_dl = 'data-bocc-pro-landing'
    generation_date = generation_date = str(datetime.now())[0:10]
    #generation_date = '2021-02-22'
    num_rows = str(len(df.axes[0])+1)
    size = str(os.path.getsize(file_path))
    version = '6-0'
    meta_file = domain + '_' + subdomain + '_' + source + '_v' + version + '.meta'  
      
    if (first_letter == "D" and len_period == 9): # Diario Incremental
        start_period = periodo[1:5] + '-' +  periodo[5:7] + '-' +  periodo[7:9]
        end_period = periodo[1:5] + '-' +  periodo[5:7] + '-' +  periodo[7:9]
        frequency = 'DAILY'
    elif (first_letter == "H" and len_period == 18): # Diario Historico
        start_period = periodo[1:5] + '-' +  periodo[5:7] + '-' +  periodo[7:9]
        end_period = periodo[10:14] + '-' +  periodo[14:16] + '-' +  periodo[16:18]
        frequency = 'HISTORICAL'
    elif (first_letter == "M" and len_period == 7): # Mensual Incremental
        start_period = periodo[1:5] + '-' +  periodo[5:7] + '-' + '01'
        end_period = periodo[1:5] + '-' +  periodo[5:7] + '-' + str(monthrange(int(periodo[1:5]), int(periodo[5:7]))[1])
        frequency = 'MONTHLY'   
    elif (first_letter == "H" and len_period == 14): # Mensual Historico
        start_period = periodo[1:5] + '-' +  periodo[5:7]
        end_period = periodo[8:12] + '-' +  periodo[12:14]
        frequency = 'HISTORICAL'
    elif (first_letter == "F" and len_period == 8): # Quincenal
        if periodo[-1:] == '1':
            start_period = periodo[1:5] + '-' +  periodo[5:7] + '-' + '01'
            end_period = periodo[1:5] + '-' +  periodo[5:7] + '-' + '15'
        elif periodo[-1:] == '2':
            start_period = periodo[1:5] + '-' +  periodo[5:7] + '-' + '16'
            end_period = periodo[1:5] + '-' +  periodo[5:7] + '-' + str(monthrange(int(periodo[1:5]), int(periodo[5:7]))[1])
        frequency = 'BIWEEKLY'  
    elif (first_letter == "Q" and len_period == 6): # Trimestral Q20204
        if periodo[-1:] == '1':
            start_period = periodo[1:5] + '-' +  '01' + '-' + '01'
            end_period = periodo[1:5] + '-' +  '03' + '-' + str(monthrange(int(periodo[1:5]), 3)[1])
        elif periodo[-1:] == '2':
            start_period = periodo[1:5] + '-' +  '04' + '-' + '01'
            end_period = periodo[1:5] + '-' +  '06' + '-' + str(monthrange(int(periodo[1:5]), 6)[1])
        elif periodo[-1:] == '3':
            start_period = periodo[1:5] + '-' +  '07' + '-' + '01'
            end_period = periodo[1:5] + '-' +  '09' + '-' + str(monthrange(int(periodo[1:5]), 9)[1])
        elif periodo[-1:] == '4':
            start_period = periodo[1:5] + '-' +  '10' + '-' + '01'
            end_period = periodo[1:5] + '-' +  '12' + '-' + str(monthrange(int(periodo[1:5]), 12)[1])     
        frequency = 'QUARTERLY'
    else:
        print("ERROR: NOMENCLATURA DE PERIODO", periodo)
      
    datos = '{\n' + '\t' + '\
"entity": ' + '"' + entity + '"' + ',\n' + '\t' + '\
"filename": ' + '"' + filename + '"' + ',\n' + '\t' + '\
"domain": ' + '"' + domain + '"' + ',\n' + '\t' + '\
"subdomain": ' + '"' + subdomain + '"' + ',\n' + '\t' + '\
"source": ' + '"' + source + '"' + ',\n' + '\t' + '\
"bucket_name_dl": ' + '"' + bucket_name_dl + '"' + ',\n' + '\t' + '\
"generation_date": ' + '"' + generation_date + '"' + ',\n' + '\t' + '\
"start_period": ' + '"' + start_period + '"' + ',\n' + '\t' + '\
"end_period": ' + '"' + end_period + '"' + ',\n' + '\t' + '\
"frequency": ' + '"' + frequency + '"' + ',\n' + '\t' + '\
"num_rows": ' + str(num_rows) + ',\n' + '\t' + '\
"size": ' + '"' + size + '"' + ',\n' + '\t' + '\
"meta_file": ' + '"' + meta_file + '"' + ',\n' + '\t' + '\
"version": ' + '"' + version + '"' + '\n' +  '\
}'
                            
    file_load = file_path[0:-4] + '.load'
    file = open(file_load,'w')
    file.write(datos)
    file.close() 
    print('\t', file_load)
    print(datos)

###### read_meta

In [49]:
def read_meta(file_path_dicc):
    df_dicc = pd.read_excel(file_path_dicc, sheet_name = 'NOMBRE_TABLA_FIELDS', skiprows = 1)
    df_dicc = df_dicc[['FIELD_NAME', 'DATA_TYPE', 'FORMAT', 'LENGTH', 'PRECISION', 'SCALE']]
    df_dicc['length'] = df_dicc['LENGTH'].astype('Int64')
    df_dicc['precision'] = df_dicc['PRECISION'].astype('Int64')
    df_dicc['SCALE'] = df_dicc.SCALE.fillna(0)
    df_dicc['scale'] = df_dicc['SCALE'].astype('Int64')
    df_dicc['type_data'] = df_dicc['DATA_TYPE']
    df_dicc['type_dict'] = np.where(df_dicc['type_data'] == 'string', 
                                    df_dicc['type_data'] + '(' + df_dicc['length'].astype(str) + ')', np.nan)
    df_dicc['type_dict'] = np.where(df_dicc['type_data'] == 'number', 
                                    df_dicc['type_data'] + '(' + df_dicc['precision'].astype(str) + ',' +  
                                    df_dicc['scale'].astype(str) + ')' , df_dicc['type_dict'])
    df_dicc['type_dict'] = np.where(df_dicc['type_data'] == 'date', 
                                    df_dicc['type_data'] + '(' + df_dicc['FORMAT'].astype(str) + ')', 
                                    df_dicc['type_dict'])
    df_dicc['type_np'] = np.where((df_dicc['DATA_TYPE'] == 'number') & (df_dicc['scale'] > 0), 'float64',
                                np.where((df_dicc['DATA_TYPE'] == 'number') & (df_dicc['scale'] == 0), 'Int64',
                                    np.where((df_dicc['DATA_TYPE'] == 'string'), 'object',
                                             'datetime64[ns]')))
    
    df_dicc['type'] = np.where((df_dicc['DATA_TYPE'] == 'number') & (df_dicc['scale'] > 0), 'float',
                                np.where((df_dicc['DATA_TYPE'] == 'number') & (df_dicc['scale'] == 0), 'integer',
                                    np.where((df_dicc['DATA_TYPE'] == 'string'), 'string',
                                             'date')))
    
    df_dicc = df_dicc[['FIELD_NAME', 'type_data','type_dict','type_np', 'type', 'length', 'precision','scale' ]]     
    df_meta = pd.read_excel(file_path_dicc, sheet_name = 'NOMBRE_TABLA_DESC', skiprows = 1)
    
    dict_meta = {}
    
    dict_meta['dominio'] = df_meta['domain'].loc[0]
    dict_meta['subdominio'] = df_meta['subdomain'].loc[0]
    dict_meta['fuente'] = df_meta['source'].loc[0]
    dict_meta['extension'] = '.' + df_meta['file_format'].loc[0]
    dict_meta['cols_string'] = df_dicc[(df_dicc['type_data'] == 'string') & 
                                       (df_dicc['scale'] == 0)].FIELD_NAME.tolist()
    dict_meta['cols_float'] = df_dicc[(df_dicc['type_data'] == 'number') & 
                                      (df_dicc['scale'] > 0)].FIELD_NAME.tolist()
    dict_meta['cols_integer'] = df_dicc[(df_dicc['type_data'] == 'number') & 
                                        (df_dicc['scale'] == 0)].FIELD_NAME.tolist()
    dict_meta['cols_date'] = df_dicc[df_dicc['type_data'] == 'date'].FIELD_NAME.tolist()
    dict_meta['cols'] = df_dicc.set_index('FIELD_NAME').T.to_dict('dict')  
    
    dict_meta['list_cols'] = []
   

    for i in df_dicc.index:
        dict_row = {'name': df_dicc['FIELD_NAME'][i], 
                    'type_np': df_dicc['type_np'][i], 
                    'type_dict': df_dicc['type_dict'][i],
                    'type': df_dicc['type'][i]}
    
    return dict_meta    

###### validacion_columnas

In [50]:
def validacion_columnas(df_original): # valida que el archivo contenga las mismas columnas del diccionario
    validacion = {}
    validacion['list_cols_dict'] = list(meta['cols'].keys())
    validacion['list_cols_adicionales'] = list(set(df_original.columns) - set(validacion['list_cols_dict']))
    validacion['list_cols_faltantes'] = list(set(validacion['list_cols_dict'])  - set(df_original.columns))
    if len(validacion['list_cols_faltantes']) == 0: 
        validacion['list_cols_faltantes_check'] = True 
    else: 
        validacion['list_cols_faltantes_check'] = False
    if len(validacion['list_cols_adicionales']) == 0: 
        validacion['list_cols_adicionales_check'] = True 
    else: 
        validacion['list_cols_adicionales_check'] = False 
    return validacion

###### transformacion_datos

In [51]:
def transformacion_datos(df_original, meta):
    columns = list(meta['cols'].keys())
    try:
        df_final = df_original[columns] # WARNING
    except:
        df_final = df_original
        print('\t', 'error columna faltante - no se encuentra campo:', sys.exc_info()[1] )
#    df_final.loc[:columns] = df_original[columns]
    num_error_tipo = 0

# Cambios de Tipo - Conversión a Entero
    for campo in meta['cols_integer']:
        try:
            df_final.loc[:, campo] = pd.to_numeric(df_final.loc[:, campo], errors = 'raise')
            df_final.loc[:, campo] = df_final.loc[:, campo].astype('Int64')
        except:
            num_error_tipo = num_error_tipo + 1
            print('\t', 'error tipo de dato (integer) - no se puede convertir campo:', campo, sys.exc_info()[1])
        
# Cambios de Tipo - Conversión y Redondeo a Float
    for campo in meta['cols_float']:
        try:
            df_final.loc[:, campo] = pd.to_numeric(df_final.loc[:, campo], errors = 'raise')
            df_final.loc[:, campo] = df_final.loc[:, campo].astype('float64')
            df_final.loc[:, campo] = pd.Series([round(val,meta['cols'][campo]['scale']) for val in df_final[campo]])
        except:
            num_error_tipo = num_error_tipo + 1
            print('\t', 'error tipo de dato (float) - no se puede convertir campo:', campo, sys.exc_info()[1])
    
# Cambios de Tipo - Conversión a Fecha
    for campo in meta['cols_date']:
        try:
            df_final.loc[:, campo] = pd.to_datetime(df_final[campo], format='%Y-%m-%d %H:%M:%S')
            df_final.loc[:, campo] = df_final[campo].dt.strftime('%d/%m/%Y')
            df_final.loc[:, campo] = pd.to_datetime(df_final[campo], format='%d/%m/%Y')
        except:
            num_error_tipo = num_error_tipo + 1
            print('\t', 'error tipo de dato (date) - no se puede convertir campo:', campo, sys.exc_info()[1])
        
# Cambios de Tipo - Transformación Strings
    for campo in meta['cols_string']:
        try:
            df_final.loc[:, campo] = df_final[campo].str.replace('\n',' ')
            df_final.loc[:, campo] = df_final[campo].str.strip()
            df_final.loc[:, campo] = df_final[campo].str.replace('"','')
            df_final.loc[:, campo] = df_final[campo].str.replace('\t',' ')
            df_final.loc[:, campo] = df_final[campo].str.replace('|','¬')
        except:
            num_error_tipo = num_error_tipo + 1
            print('\t', 'error tipo de dato (string) - no se puede convertir campo:', campo, sys.exc_info()[1])
        
    if num_error_tipo > 0:
        print('\t', 'validación tipo de datos (archivo original vs meta): False', '\n')
    else:
        print('\t', 'validación tipo de datos (archivo original vs meta): True', '\n')
        
    return df_final

###### validacion_archivo_final

In [52]:
def validacion_archivo_final(df_final, df_original, meta, validacion_cols):
    dict_val_arch = {}
    list_cols_meta = validacion_cols['list_cols_dict']

#================================================================================================
#******              metricas calidad             ******
    # name
#    print('\nValidación Archivo Final......')
    name = {}
    name['pre'] = nombre_archivo
    name['post'] = '{}_{}_{}_{}{}'.format(meta['dominio'], meta['subdominio'],meta['fuente'], periodo, 
                                                       meta['extension'])
    # num_rows 
    num_rows = {}
    num_rows['pre'] = df_original.shape[0] + 1 # mas 1 por el encabezado
    num_rows['post'] = df_final.shape[0] + 1 # mas 1 por el encabezado
    num_rows['check'] = num_rows['pre'] == num_rows['post']

    # num_cols
    num_cols = {}
    num_cols['pre'] = df_original.shape[1] 
    num_cols['post'] = df_final.shape[1] 
    num_cols['num_cols_faltantes_check'] = validacion_cols['list_cols_faltantes_check']
    num_cols['list_cols_faltantes'] = validacion_cols['list_cols_faltantes']
    num_cols['num_cols_adicionales_check'] = num_cols['pre'] == num_cols['post']
    num_cols['list_cols_adicionales'] = validacion_cols['list_cols_adicionales'] 
    
    # check
    check = {}
    check['num_rows_check'] = num_rows['pre'] == num_rows['post']
    check['num_cols_faltantes_check'] = validacion_cols['list_cols_faltantes_check']

#================================================================================================
#******       metricas calidad por columna  (conteos, nulos, tipos)        ******
    cols = {}
    cols_false = 0

    for col in list_cols_meta:
        try:    
            if df_final[col].dtype == 'object': # df_final[col].dtype
                tipo = 'string'
            elif df_final[col].dtype == 'Int64': #WARNING
                tipo = 'integer'
            elif df_final[col].dtype == 'float64':
                tipo = 'float'
            else: 
                tipo = 'date'

            list_col_val = {}

            list_col_val['nulls'] = {}
            list_col_val['nulls']['pre'] =  df_original[col].isnull().sum() 
            list_col_val['nulls']['post'] =  df_final[col].isnull().sum()
            nulls_check = list_col_val['nulls']['pre'] == list_col_val['nulls']['post'] 

            list_col_val['count'] = {}
            list_col_val['count']['pre'] =  df_original[col].count() 
            list_col_val['count']['post'] =  df_final[col].count()
            count_check = list_col_val['count']['pre'] == list_col_val['count']['post']

            list_col_val['types'] = {}
            list_col_val['types']['pre'] = meta['cols'][col]['type']
            list_col_val['types']['post'] = tipo     
            types_check = df_final[col].dtype == meta['cols'][col]['type_np'] 

            if nulls_check == True and count_check == True and types_check:
                list_col_val['check'] = True
            else:
                list_col_val['check'] = False

            cols[col] = list_col_val

    #******       validación columnas         ******    
            if cols[col]['check'] == False:
                cols_false = cols_false + 1
                #se puede crear una lista y agregar las columnas False
        except:
            list_col_val['check'] = False
            cols_false = cols_false + 1
            #se puede crear una lista y agregar las columnas False

    if cols_false > 0:
        cols_check = False
    else:
        cols_check = True

    cols['cols_check'] = cols_check
    print('\t','errores encontrados:', cols_false, '\n')
    check['cols_check'] = cols_check
    #================================================================================================
    #******              log final             ******
    # log final
    dict_val_arch['name'] = name
    dict_val_arch['check'] = check
    dict_val_arch['num_rows'] = num_rows
    dict_val_arch['num_cols'] = num_cols
    dict_val_arch['cols'] = cols
    dict_val_arch
    #================================================================================================
    return dict_val_arch

###### write_json

In [53]:
def write_json(meta, dict_val_arch, periodo):
    json_path = path + meta['dominio'] + '_' + meta['subdominio'] + '_' + meta['fuente'] + '_' + periodo + '.json' 
    json_path_2 = 'D:/Fuentes/Planos/Logs/' + meta['dominio'] + '_' + meta['subdominio'] + '_' + meta['fuente'] + '_' + periodo + '.json' 
    archivo = dict_val_arch

    class NpEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, np.integer):
                return int(obj)
            elif isinstance(obj, np.floating):
                return float(obj)
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, np.bool_):             
                return bool(obj)
            else:
                return super(NpEncoder, self).default(obj)
# Your codes .... 
    x = json.dumps(archivo, cls=NpEncoder)

    with open(json_path, 'w') as fp:
        json.dump(archivo, fp, cls=NpEncoder)
    print('\t', json_path, '\n')
    with open(json_path_2, 'w') as fp:
        json.dump(archivo, fp, cls=NpEncoder)

### Proceso Final

Objetos de entrada:
- validacion_cols (diccionario que devuelve la función 'validacion_columnas')
- meta (diccionario que devuelve la función 'read_meta')
- df_original (dataframe que devuelve la función 'read_file')
- df_final (dataframe que devuelve la función 'transformacion_datos')


In [54]:
print('Inicio Ejecución......') 
print('\t', datetime.now(), '\n')

#******  cargue de meadata necesaria (dict_meta)  ******
print('Cargue Archivos......')
meta = read_meta(file_path_dict)
print('\t', 'meta (diccionario): True')

#******  cargue archivo inicial/original  ******
df_original = read_file(file_path)
print('\t', 'archivo original: True', '\n')

#******  validación columnas ******
print('Validación Columnas......')
validacion_cols = validacion_columnas(df_original)
print('\t', 'validación columnas faltantes: ',validacion_cols['list_cols_faltantes_check'] )
print('\t', 'lista columnas faltantes: ',validacion_cols['list_cols_faltantes'] )
print('\t', 'lista columnas adicionales: ',validacion_cols['list_cols_adicionales'], '\n')

#******  transformación datos  ****** !!!!!REVISAR WARNINGS!!!!!
print('Validación Datos......')
pd.options.mode.chained_assignment = None  # default='warn' - desactiva warning
df_final = transformacion_datos(df_original, meta)
pd.options.mode.chained_assignment = 'warn'  # default='warn' - activa warning

#******  validación datos  ****** 
print('Validación Final......')
dict_val_arch = validacion_archivo_final(df_final, df_original, meta, validacion_cols)

#******  validación archivo final ****** 
check_false = 0
for x in dict_val_arch['check']:
    if dict_val_arch['check'][x] == False:
        check_false = check_false + 1
        #print('\t', 'error en:', x)

#******  export archivo final, log y load ****** 
if check_false == 0:
    print('Generación Archivo Final......')
    file_export = path + dict_val_arch['name']['post']
    print('\t', file_export, '\n')
    df_final.to_csv(file_export, sep = '|', encoding = 'utf-8', 
                decimal = ',', doublequote = False, index = False, date_format='%d/%m/%Y')
    
    print('Generación Log......')
    write_json(meta, dict_val_arch, periodo)
    print('Generación Load......')
    generar_load(file_export)
else:
    print('Generación Log......')
    write_json(meta, dict_val_arch, periodo)
    print('============= No se genera archivo =============')
df_final

Inicio Ejecución......
	 2024-11-22 08:08:42.384548 

Cargue Archivos......
	 meta (diccionario): True
	 archivo original: True 

Validación Columnas......
	 validación columnas faltantes:  True
	 lista columnas faltantes:  []
	 lista columnas adicionales:  ['na2_obl', 'na1_com_ml', 'na1_cant_obl_leas', 'na3_cant_obl_com', 'na3_saldo_total', 'na3_cant_obl_leas', 'na2_cant_obl_leas', 'na3_com_me', 'na1_saldo_total', 'na3_com_ml', 'na1_cant_obl_com', 'na1_obl', 'na2_cant_obl_com', 'na2_saldo_total', 'unnamed_472', 'na1_com_me', 'na2_com_me', 'na3_obl', 'na2_com_ml', 'unnamed_147'] 

Validación Datos......


<ipython-input-51-f5131ab5f393>:47: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_final.loc[:, campo] = df_final[campo].str.replace('|','¬')


	 validación tipo de datos (archivo original vs meta): True 

Validación Final......
	 errores encontrados: 0 

Generación Archivo Final......
	 D:/Fuentes/Planos/Cifin-Empresas/riesgos-creditos_cifin-empresas_cifin_Q20242.csv 

Generación Log......
	 D:/Fuentes/Planos/Cifin-Empresas/riesgos-creditos_cifin-empresas_cifin_Q20242.json 

Generación Load......
	 D:/Fuentes/Planos/Cifin-Empresas/riesgos-creditos_cifin-empresas_cifin_Q20242.load
{
	"entity": "BOCC",
	"filename": "riesgos-creditos_cifin-empresas_cifin_Q20242.csv",
	"domain": "riesgos-creditos",
	"subdomain": "cifin-empresas",
	"source": "cifin",
	"bucket_name_dl": "data-bocc-pro-landing",
	"generation_date": "2024-11-22",
	"start_period": "2024-04-01",
	"end_period": "2024-06-30",
	"frequency": "QUARTERLY",
	"num_rows": 27361,
	"size": "55724063",
	"meta_file": "riesgos-creditos_cifin-empresas_cifin_v6-0.meta",
	"version": "6-0"
}


,area,llave,vicepresidente,gerente_zonal,cod_gte,nombre_gerente,ciudad_gerente_listado,ciudad_gerente,cod_ciudad,departamento,region,ciudad_gerente2,marca_ciudad_gte,nit,razon,seg_gte,seg_gte_foco,seg_cli,clasificacion_del_cliente,ciudad_cliente,banco_de_occidente_com_ml,banco_de_occidente_com_me,banco_de_occidente_saldo_total,bancolombia_com_ml,bancolombia_com_me,bancolombia_saldo_total,de_bogota_com_ml,de_bogota_com_me,de_bogota_saldo_total,itau_com_ml,itau_com_me,itau_saldo_total,davivienda_com_ml,davivienda_com_me,davivienda_saldo_total,bbva_com_ml,bbva_com_me,bbva_saldo_total,popular_com_ml,popular_com_me,popular_saldo_total,scotiabank_colpatria_com_ml,scotiabank_colpatria_com_me,scotiabank_colpatria_saldo_total,banagrario_com_ml,banagrario_com_me,banagrario_saldo_total,av_villas_com_ml,av_villas_com_me,av_villas_saldo_total,gnb_sudameris_com_ml,gnb_sudameris_com_me,gnb_sudameris_saldo_total,citibank_com_ml,citibank_com_me,citibank_saldo_total,bcsc_com_ml,bcsc_com_me,bcsc_saldo_total,bancoomeva_com_ml,bancoomeva_com_me,bancoomeva_saldo_total,banco_pichincha_com_ml,banco_pichincha_com_me,banco_pichincha_saldo_total,santander_com_ml,santander_com_me,santander_saldo_total,multibank_com_ml,multibank_com_me,multibank_saldo_total,banco_procredit_com_ml,banco_procredit_com_me,banco_procredit_saldo_total,wwb_com_ml,wwb_com_me,wwb_saldo_total,banco_finandina_com_ml,banco_finandina_com_me,banco_finandina_saldo_total,bancamia_com_ml,bancamia_com_me,bancamia_saldo_total,mundo_mujer_com_ml,mundo_mujer_com_me,mundo_mujer_saldo_total,leasing_bancoldex_com_ml,leasing_bancoldex_com_me,leasing_bancoldex_saldo_total,findeter_com_ml,findeter_com_me,findeter_saldo_total,idea_com_ml,idea_com_me,idea_saldo_total,coopcentral_com_ml,coopcentral_com_me,coopcentral_saldo_total,serfinanza_com_ml,serfinanza_com_me,serfinanza_saldo_total,coltefinanciera_com_ml,coltefinanciera_com_me,coltefinanciera_saldo_total,leasing_corficol_com_ml,leasing_corficol_com_me,leasing_corficol_saldo_total,gmac_com_ml,gmac_com_me,gmac_saldo_total,dann_regional_com_ml,dann_regional_com_me,dann_regional_saldo_total,fin_juriscoop_com_ml,fin_juriscoop_com_me,fin_juriscoop_saldo_total,fin_des_nacional_com_ml,fin_des_nacional_com_me,fin_des_nacional_saldo_total,confiar_com_ml,confiar_com_me,confiar_saldo_total,fna_com_ml,fna_com_me,fna_saldo_total,alpopular_com_ml,alpopular_com_me,alpopular_saldo_total,cotrafa_com_ml,cotrafa_com_me,cotrafa_saldo_total,cfa_com_ml,cfa_com_me,cfa_saldo_total,crediseguros_com_ml,crediseguros_com_me,crediseguros_saldo_total,intercorp_com_ml,intercorp_com_me,intercorp_saldo_total,reintegra_com_ml,reintegra_com_me,reintegra_saldo_total,fiducolombia_com_ml,fiducolombia_com_me,fiducolombia_saldo_total,maicito_com_ml,maicito_com_me,maicito_saldo_total,coofinep_com_ml,coofinep_com_me,coofinep_saldo_total,bancompartir_com_ml,bancompartir_com_me,bancompartir_saldo_total,covinoc_com_ml,covinoc_com_me,covinoc_saldo_total,fonade_com_ml,fonade_com_me,fonade_saldo_total,fiducentral_com_ml,fiducentral_com_me,fiducentral_saldo_total,rf_soluciones_com_ml,rf_soluciones_com_me,rf_soluciones_saldo_total,comiagro_com_ml,comiagro_com_me,comiagro_saldo_total,refinancia_com_ml,refinancia_com_me,refinancia_saldo_total,opport_int_com_ml,opport_int_com_me,opport_int_saldo_total,fidu_corficol_com_ml,fidu_corficol_com_me,fidu_corficol_saldo_total,sistemcobro_com_ml,sistemcobro_com_me,sistemcobro_saldo_total,giros_finanzas_com_ml,giros_finanzas_com_me,giros_finanzas_saldo_total,promotora_inv_com_ml,promotora_inv_com_me,promotora_inv_saldo_total,fidu_previsora_com_ml,fidu_previsora_com_me,fidu_previsora_saldo_total,b_inv_bancol_com_ml,b_inv_bancol_com_me,b_inv_bancol_saldo_total,aecsa_com_ml,aecsa_com_me,aecsa_saldo_total,fideicom_rai_com_ml,fideicom_rai_com_me,fideicom_rai_saldo_total,fide_millenium_com_ml,fide_millenium_com_me,fide_millenium_saldo_total,john_f_kennedy_com_ml,john_f_kennedy_com_me,john_f_kennedy_saldo_total,banco_caja_social_com_ml,banco_caja_social_com_me,banc